In [1]:
from sqlalchemy import create_engine
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, insert
import os


In [2]:
azure_client_id = os.getenv("AZURE_CLIENT_ID")
azure_client_secret = os.getenv("AZURE_CLIENT_SECRET")
azure_tenant_id = os.getenv("AZURE_TENANT_ID")
server = os.getenv("SERVER")
database = os.getenv("DATABASE")
driver="ODBC Driver 18 for SQL Server"
connection_string = (f"mssql+pyodbc://{azure_client_id}:{azure_client_secret}@{server}:1433/{database}"
f"?driver={driver}"
"&authentication=ActiveDirectoryServicePrincipal"
"&timeout=120"
"&Encrypt=yes"
"&TrustServerCertificate=no"
)

In [ ]:
engine2 = create_engine(connection_string, connect_args={'timeout': 120})


In [41]:
from azure.identity import ClientSecretCredential
from azure.storage.filedatalake import FileSystemClient
import pyodbc, struct
import os
from msal import ConfidentialClientApplication
azure_client_id = os.getenv("AZURE_CLIENT_ID")
azure_client_secret = os.getenv("AZURE_CLIENT_SECRET")
azure_tenant_id = os.getenv("AZURE_TENANT_ID")
server = os.getenv("SERVER")
database = os.getenv("DATABASE")
driver="ODBC Driver 18 for SQL Server"
AUTHORITY = f"https://login.microsoftonline.com/{azure_tenant_id}"
connection_string = f'Driver={driver};Server=tcp:{server},1433;Database={database};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30'
credential = ClientSecretCredential(tenant_id=azure_tenant_id,
                              client_id=azure_client_id,
                              client_secret=azure_client_secret)

token_bytes = credential.get_token("https://database.windows.net/.default").token.encode("UTF-16-LE")
token_struct = struct.pack(f'<I{len(token_bytes)}s', len(token_bytes), token_bytes)
SQL_COPT_SS_ACCESS_TOKEN = 1256  # This connection option is defined by microsoft in msodbcsql.h
conn = pyodbc.connect(connection_string, attrs_before={1256: token_struct})
cursor = conn.cursor()
cursor.execute("SELECT * FROM [dbo].[new_table] WHERE name = 'Test User'")

row = cursor.fetchall()
for ro in row:
    print(f"{ro.age}, {ro.name}")
# cursor.execute("INSERT INTO [dbo].[new_table] (age, name) VALUES (25, 'Test User')")
# conn.commit()

25, Test User


In [49]:
from sqlmodel import SQLModel, Session, Field, create_engine, select
from azure.identity import ClientSecretCredential
import os
import struct

# --- Azure credentials ---
azure_client_id = os.getenv("AZURE_CLIENT_ID")
azure_client_secret = os.getenv("AZURE_CLIENT_SECRET")
azure_tenant_id = os.getenv("AZURE_TENANT_ID")
server = os.getenv("SERVER")     # full Fabric/SQL hostname, e.g. "xxx.sql.azuredatabases.net"
database = os.getenv("DATABASE")

# --- Get access token ---
credential = ClientSecretCredential(
    tenant_id=azure_tenant_id,
    client_id=azure_client_id,
    client_secret=azure_client_secret
)
token = credential.get_token("https://database.windows.net/.default").token
token_bytes = token.encode("utf-16-le")  # utf-16-le = required by SQL Server
token_struct = struct.pack(f"<I{len(token_bytes)}s", len(token_bytes), token_bytes)

# --- SQLAlchemy engine using ODBC Driver ---
odbc_str = (
    f"mssql+pyodbc://@{server}:1433/{database}"
    "?driver=ODBC+Driver+18+for+SQL+Server"
    "&Encrypt=yes&TrustServerCertificate=no&Connection+Timeout=30"
)

from sqlalchemy import event

engine = create_engine(odbc_str)

# Inject access token on connect
@event.listens_for(engine, "do_connect")
def provide_token(dialect, conn_rec, cargs, cparams):
    cparams["attrs_before"] = {1256: token_struct}  # 1256 = SQL_COPT_SS_ACCESS_TOKEN

# --- Define SQLModel (example) ---
class User(SQLModel, table=True):
    __tablename__ = "new_table"
    __table_args__ = {"schema": "dbo","extend_existing": True}
    age: int | None = Field(default=None, primary_key=True)
    name: str | None = None

# --- Query with SQLModel ---
SQLModel.metadata.create_all(engine)

with Session(engine) as session:
    stmt = select(User).where(User.name == "Test User")
    results = session.exec(stmt).all()
    for row in results:
        print(row)


DBAPIError: (pyodbc.Error) ('FA005', '[FA005] [Microsoft][ODBC Driver 18 for SQL Server]Cannot use Access Token with any of the following options: Authentication, Integrated Security, User, Password. (0) (SQLDriverConnect); [FA005] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [FA005] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [FA005] [Microsoft][ODBC Driver 18 for SQL Server]Cannot use Access Token with any of the following options: Authentication, Integrated Security, User, Password. (0)')
(Background on this error at: https://sqlalche.me/e/20/dbapi)

In [78]:
from sqlmodel import SQLModel, Session, select
from azure.identity import ClientSecretCredential
from sqlalchemy import create_engine, event
import os, struct

# --- Azure creds ---
azure_client_id = os.getenv("AZURE_CLIENT_ID")
azure_client_secret = os.getenv("AZURE_CLIENT_SECRET")
azure_tenant_id = os.getenv("AZURE_TENANT_ID")
server = os.getenv("SERVER")     # e.g. "xxx.datawarehouse.fabric.microsoft.com"
database = os.getenv("DATABASE")

# --- Get access token ---
credential = ClientSecretCredential(
    tenant_id=azure_tenant_id,
    client_id=azure_client_id,
    client_secret=azure_client_secret,
)
token = credential.get_token("https://database.windows.net/.default").token
token_bytes = token.encode("utf-16-le")
token_struct = struct.pack(f"<I{len(token_bytes)}s", len(token_bytes), token_bytes)

# --- Connection string (NO Authentication here!) ---
# odbc_str = (
#     f"mssql+pyodbc://@{server}:1433/{database}"
#     "?driver=ODBC+Driver+18+for+SQL+Server"
#     "&Encrypt=yes&TrustServerCertificate=no&Connection+Timeout=30"
# )
# odbc_str = (f"mssql+pyodbc://{azure_client_id}:{azure_client_secret}@{server}:1433/{database}"
# f"?driver={driver}"
# "&authentication=ActiveDirectoryServicePrincipal"
# "&timeout=120"
# "&Encrypt=yes"
# "&TrustServerCertificate=no"
# "&Authentication=ActiveDirectoryAccessToken"
# )
import urllib
driver_name = '{ODBC Driver 18 for SQL Server}'
server_name = os.getenv("SERVER")
database_name = os.getenv("DATABASE")
connection_string = 'Driver={};Server=tcp:{}.database.windows.net,1433;Database={};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30'.format(driver_name, server_name, database_name)

params = urllib.parse.quote(connection_string)
odbc_str = "mssql+pyodbc:///?odbc_connect={0}".format(params)

# --- Create engine ---
engine = create_engine(odbc_str, echo=True)
# engine = create_engine(odbc_str, echo=True, connect_args = {"attrs_before":{1256: token_struct}})
# --- Inject the access token ---
@event.listens_for(engine, "do_connect")
def provide_token(dialect, conn_rec, cargs, cparams):
    cparams["attrs_before"] = {1256: token_struct}  # SQL_COPT_SS_ACCESS_TOKEN

# # --- Example table model ---
# from sqlmodel import Field

class New_table(SQLModel, table=True):
    __tablename__ = "new_table"
    __table_args__ = {"schema": "dbo", "extend_existing": True}

    age: int | None = Field(default=None, primary_key=True)
    name: str | None = None


new_user = New_table(age=30, name="You")

with Session(engine) as session:
    session.add(new_user)
    session.commit()
# --- Query ---
with Session(engine) as session:
    stmt = select(New_table)
    results = session.exec(stmt).all()
    for row in results:
        print(row)


OperationalError: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 18 for SQL Server]TCP Provider: Kjenner ingen slik vert.\r\n (11001) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to tcp:b5li4afpbdwezicwwnkeoubzse-m3nesqac3ueepkmopydvg35o3y.datawarehouse.fabric.microsoft.com.database.windows.net,1433. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (11001)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [84]:
import os
import struct
import urllib
from sqlmodel import SQLModel, Field, Session, select
from sqlalchemy import create_engine, event
from azure.identity import DefaultAzureCredential

# --- Azure creds ---
driver_name = "{ODBC Driver 18 for SQL Server}"
server_name = os.getenv("SERVER")      # e.g. "xxx.datawarehouse.fabric.microsoft.com"
database_name = os.getenv("DATABASE")

# --- Connection string (NO auth here, token comes later) ---
connection_string = (
    "Driver={};Server=tcp:{},1433;Database={};Encrypt=yes;"
    "TrustServerCertificate=no;Connection Timeout=30"
).format(driver_name, server_name, database_name)

params = urllib.parse.quote(connection_string)
odbc_str = f"mssql+pyodbc:///?odbc_connect={params}"

# --- Get SQLAlchemy engine ---
engine = create_engine(odbc_str, echo=True)

# --- Token injection hook ---
credential = DefaultAzureCredential()  # or ClientSecretCredential if you prefer

@event.listens_for(engine, "do_connect")
def provide_token(dialect, conn_rec, cargs, cparams):
    print("🔑 Requesting new Entra ID token for SQL Server...")
    token_bytes = credential.get_token("https://database.windows.net/.default").token.encode("utf-16-le")
    token_struct = struct.pack(f"<I{len(token_bytes)}s", len(token_bytes), token_bytes)
    SQL_COPT_SS_ACCESS_TOKEN = 1256
    cparams["attrs_before"] = {SQL_COPT_SS_ACCESS_TOKEN: token_struct}

# --- SQLModel table ---
class new_table(SQLModel, table=True):
    __tablename__ = "new_table"
    __table_args__ = {"schema": "dbo", "extend_existing": True}

    age: int | None = Field(default=None, primary_key=True)
    name: str | None = None

# --- Create tables only if needed (optional) ---
# SQLModel.metadata.create_all(engine)

# --- Insert example ---
new_user = new_table(age=30, name="You")

with Session(engine) as session:
    session.add(new_user)
    session.commit()
# with Session(engine) as session:
#     new_user = new_table(name="You")   # let SQL Server assign age
#     session.add(new_user)
#     session.commit()
#     session.refresh(new_user)  # fetch the generated ID
#     print(new_user.age, new_user.name)

# --- Query example ---
with Session(engine) as session:
    stmt = select(new_table)
    results = session.exec(stmt).all()
    for row in results:
        print(f"✅ Row: age={row.age}, name={row.name}")


🔑 Requesting new Entra ID token for SQL Server...
2025-09-22 12:03:23,771 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2025-09-22 12:03:23,772 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-22 12:03:23,795 INFO sqlalchemy.engine.Engine SELECT schema_name()
2025-09-22 12:03:23,796 INFO sqlalchemy.engine.Engine [generated in 0.00114s] ()
2025-09-22 12:03:23,837 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2025-09-22 12:03:23,838 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()
2025-09-22 12:03:23,850 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2025-09-22 12:03:23,851 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ()
2025-09-22 12:03:23,878 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-22 12:03:23,887 INFO sqlalchemy.engine.Engine INSERT INTO dbo.new_table (age, name) VALUES (?, ?)
2025-09-22 12:03:23

C:\Users\IGNTOR\AppData\Local\Temp\ipykernel_18104\3658059914.py:52: SAWarning: Column 'dbo.new_table.id' is marked as a member of the primary key for table 'dbo.new_table', but has no Python-side or server-side default generator indicated, nor does it indicate 'autoincrement=True' or 'nullable=True', and no explicit value is passed.  Primary key columns typically may not store NULL. Note that as of SQLAlchemy 1.1, 'autoincrement=True' must be indicated explicitly for composite (e.g. multicolumn) primary keys if AUTO_INCREMENT/SERIAL/IDENTITY behavior is expected for one of the columns in the primary key. CREATE TABLE statements are impacted by this change as well on most backends.
  session.commit()


2025-09-22 12:03:25,692 INFO sqlalchemy.engine.Engine COMMIT
2025-09-22 12:03:25,804 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-22 12:03:25,805 INFO sqlalchemy.engine.Engine SELECT dbo.new_table.age, dbo.new_table.name, dbo.new_table.id 
FROM dbo.new_table
2025-09-22 12:03:25,806 INFO sqlalchemy.engine.Engine [generated in 0.00061s] ()
2025-09-22 12:03:25,819 INFO sqlalchemy.engine.Engine ROLLBACK


ProgrammingError: (pyodbc.ProgrammingError) ('42S22', "[42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name 'id'. (207) (SQLExecDirectW)")
[SQL: SELECT dbo.new_table.age, dbo.new_table.name, dbo.new_table.id 
FROM dbo.new_table]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [74]:
import urllib
driver_name = '{ODBC Driver 18 for SQL Server}'
server_name = os.getenv("SERVER")
database_name = os.getenv("DATABASE")
connection_string = 'Driver={};Server=tcp:{}.database.windows.net,1433;Database={};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30'.format(driver_name, server_name, database_name)

params = urllib.parse.quote(connection_string)
url = "mssql+pyodbc:///?odbc_connect={0}".format(params)
engine = create_engine(url)

In [77]:
url

'mssql+pyodbc:///?odbc_connect=Driver%3D%7BODBC%20Driver%2018%20for%20SQL%20Server%7D%3BServer%3Dtcp%3Ab5li4afpbdwezicwwnkeoubzse-m3nesqac3ueepkmopydvg35o3y.datawarehouse.fabric.microsoft.com.database.windows.net%2C1433%3BDatabase%3DPortfolioWarehouse%3BEncrypt%3Dyes%3BTrustServerCertificate%3Dno%3BConnection%20Timeout%3D30'

In [ ]:
import pyodbc
from sqlalchemy import create_engine
from azure.identity import DefaultAzureCredential

# Get an access token using DefaultAzureCredential
credential = DefaultAzureCredential()
token = credential.get_token("https://database.windows.net/.default").token

# Construct the connection string using the access token
# Note: You might need to adjust the driver and server details based on your setup.
# The 'Authentication=ActiveDirectoryAccessToken' is crucial for AD token usage.
connection_string = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Authentication=ActiveDirectoryAccessToken;"
)

# Create the SQLAlchemy engine, passing the token in connect_args
# The token needs to be passed as a byte string for pyodbc.
engine = create_engine(
    f"mssql+pyodbc:///?odbc_connect={pyodbc.quote_plus(connection_string)}",
    connect_args={"attrs_before": {pyodbc.SQL_COPT_SS_ACCESS_TOKEN: token.encode('utf-16-le')}}
)

# You can now use the engine to interact with your Azure SQL Database
# For example, to create a session:
# from sqlalchemy.orm import sessionmaker
# Session = sessionmaker(bind=engine)
# session = Session()

In [ ]:
pyodbc

In [56]:
engine = create_engine(odbc_str, echo=True, connect_args = {"attrs_before":{1256: token_struct}})

In [58]:
engine.connect()

DBAPIError: (pyodbc.Error) ('FA005', '[FA005] [Microsoft][ODBC Driver 18 for SQL Server]Cannot use Access Token with any of the following options: Authentication, Integrated Security, User, Password. (0) (SQLDriverConnect); [FA005] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [FA005] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [FA005] [Microsoft][ODBC Driver 18 for SQL Server]Cannot use Access Token with any of the following options: Authentication, Integrated Security, User, Password. (0)')
(Background on this error at: https://sqlalche.me/e/20/dbapi)

In [25]:
result["access_token"].encode("UTF-16-LE")

b'e\x00y\x00J\x000\x00e\x00X\x00A\x00i\x00O\x00i\x00J\x00K\x00V\x001\x00Q\x00i\x00L\x00C\x00J\x00h\x00b\x00G\x00c\x00i\x00O\x00i\x00J\x00S\x00U\x00z\x00I\x001\x00N\x00i\x00I\x00s\x00I\x00n\x00g\x001\x00d\x00C\x00I\x006\x00I\x00k\x00p\x00Z\x00a\x00E\x00F\x00j\x00V\x00F\x00B\x00N\x00W\x00l\x009\x00M\x00W\x00D\x00Z\x00E\x00Q\x00m\x00x\x00P\x00V\x001\x00E\x003\x00S\x00G\x004\x00w\x00T\x00m\x00V\x00Y\x00R\x00S\x00I\x00s\x00I\x00m\x00t\x00p\x00Z\x00C\x00I\x006\x00I\x00k\x00p\x00Z\x00a\x00E\x00F\x00j\x00V\x00F\x00B\x00N\x00W\x00l\x009\x00M\x00W\x00D\x00Z\x00E\x00Q\x00m\x00x\x00P\x00V\x001\x00E\x003\x00S\x00G\x004\x00w\x00T\x00m\x00V\x00Y\x00R\x00S\x00J\x009\x00.\x00e\x00y\x00J\x00h\x00d\x00W\x00Q\x00i\x00O\x00i\x00J\x00o\x00d\x00H\x00R\x00w\x00c\x00z\x00o\x00v\x00L\x002\x00F\x00w\x00a\x00S\x005\x00m\x00Y\x00W\x00J\x00y\x00a\x00W\x00M\x00u\x00b\x00W\x00l\x00j\x00c\x00m\x009\x00z\x00b\x002\x00Z\x000\x00L\x00m\x00N\x00v\x00b\x00S\x00I\x00s\x00I\x00m\x00l\x00z\x00c\x00y\x00I\x006\x00I\x00m\x00h\x

In [22]:
credential.get_token("https://database.windows.net/.default").token

'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSIsImtpZCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSJ9.eyJhdWQiOiJodHRwczovL2RhdGFiYXNlLndpbmRvd3MubmV0IiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMDA4ZTU2MGYtMDhhZi00Y2VjLWEwNTYtYjM1NDQ3NTAzOTkxLyIsImlhdCI6MTc1ODI3Njg3MywibmJmIjoxNzU4Mjc2ODczLCJleHAiOjE3NTgyODA3NzMsImFpbyI6ImsyUmdZRkJ4dTV0cDhrK2RJMW1SWGVQQ1luTTFBQT09IiwiYXBwaWQiOiI4NGRiYzc3Yy02MTMxLTQ4MmMtYTk5OC1mMzhjNGY4ZDgwZDQiLCJhcHBpZGFjciI6IjEiLCJncm91cHMiOlsiOTBjZTU1ZTAtYjliYi00ZGU5LTk5NDItZDNjYTY2YTViMDU1Il0sImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzAwOGU1NjBmLTA4YWYtNGNlYy1hMDU2LWIzNTQ0NzUwMzk5MS8iLCJpZHR5cCI6ImFwcCIsIm9pZCI6IjI4YThlNDBlLTc5NmUtNGY4NC1iZmE0LTJlMDcxZjE1MmEzMSIsInJoIjoiMS5BVGtBRDFhT0FLOEk3RXlnVnJOVVIxQTVrZE1IS1FJYkRfZEl1dHdicHF1cmJXYXFBUUE1QUEuIiwic3ViIjoiMjhhOGU0MGUtNzk2ZS00Zjg0LWJmYTQtMmUwNzFmMTUyYTMxIiwidGlkIjoiMDA4ZTU2MGYtMDhhZi00Y2VjLWEwNTYtYjM1NDQ3NTAzOTkxIiwidXRpIjoieHczS0xwcHpZMGFvYmtNR1BXRjFBQSIsInZlciI6IjEuMCIsInhtc19mdGQiOi

In [18]:
credential.get_token("https://api.fabric.microsoft.com/.default").token

'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSIsImtpZCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSJ9.eyJhdWQiOiJodHRwczovL2FwaS5mYWJyaWMubWljcm9zb2Z0LmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzAwOGU1NjBmLTA4YWYtNGNlYy1hMDU2LWIzNTQ0NzUwMzk5MS8iLCJpYXQiOjE3NTgyNzY1NDEsIm5iZiI6MTc1ODI3NjU0MSwiZXhwIjoxNzU4MjgwNDQxLCJhaW8iOiJrMlJnWUpEczFwNXgzek9JUjczRmJhdkJBM1lIQUE9PSIsImFwcGlkIjoiODRkYmM3N2MtNjEzMS00ODJjLWE5OTgtZjM4YzRmOGQ4MGQ0IiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMDA4ZTU2MGYtMDhhZi00Y2VjLWEwNTYtYjM1NDQ3NTAzOTkxLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiMjhhOGU0MGUtNzk2ZS00Zjg0LWJmYTQtMmUwNzFmMTUyYTMxIiwicmgiOiIxLkFUa0FEMWFPQUs4STdFeWdWck5VUjFBNWtRa0FBQUFBQUFBQXdBQUFBQUFBQUFDcUFRQTVBQS4iLCJzdWIiOiIyOGE4ZTQwZS03OTZlLTRmODQtYmZhNC0yZTA3MWYxNTJhMzEiLCJ0aWQiOiIwMDhlNTYwZi0wOGFmLTRjZWMtYTA1Ni1iMzU0NDc1MDM5OTEiLCJ1dGkiOiJabWZVMnZJeWZrQ2JtME1zMjl0d0FBIiwidmVyIjoiMS4wIiwieG1zX2Z0ZCI6Ik9RMW1KdGZEUEJ1R0NWVUxBcmF3TXh1TFh5T1BKS0lZVGYzLXhUcDBCeWNCYz

In [ ]:
("INSERT INTO [dbo].[new_table] (age, name) VALUES (25, 'Test User')")

In [3]:
from azure.identity import ClientSecretCredential
from azure.storage.filedatalake import FileSystemClient

cred = ClientSecretCredential(tenant_id=azure_tenant_id,
                              client_id=azure_client_id,
                              client_secret=azure_client_secret)

file_system_client = FileSystemClient(
    account_url="https://onelake.dfs.fabric.microsoft.com",
    file_system_name="Project-portfolio",
    credential=cred)
paths = file_system_client.get_paths(path="/ProjectPortfolio.Lakehouse/Tables/")
for p in paths:
    print(p.name)

/ProjectPortfolio.Lakehouse/Tables/List_portfolio
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/36456da7f1414fef9e91d453e86ed50e.parquet
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/37bcc1a3c0c44d5db0bd051f51b33b57.parquet
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/5da9babcead1438da4c5ac088603611e.parquet
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log/00000000000000000000.json
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log/00000000000000000001.json
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log/00000000000000000002.json
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log/_mashup_temporary
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log/_mashup_temporary/_mashup_temporary_9fd6ce13-1c95-4abe-b7a9-3ff5a0ba37c7.version
/ProjectPortfolio.Lakehouse/Tables/List_portfolio/_delta_log/_mashup_temporary/_mashup_temporary_b47d4639-d8f5-4fee-8ff8-c91251291059.

In [7]:
SQL_COPT_SS_ACCESS_TOKEN = 1256
def inject_azure_credential(credential, engine, token_url='https://database.windows.net/'):
    @event.listens_for(engine, 'do_connect')
    def do_connect(dialect, conn_rec, cargs, cparams):
        token = credential.get_token(token_url).token.encode('utf-16-le')
        token_struct = struct.pack(f'=I{len(token)}s', len(token), token)
        attrs_before = cparams.setdefault('attrs_before', {})
        attrs_before[SQL_COPT_SS_ACCESS_TOKEN] = bytes(token_struct)
        return dialect.connect(*cargs, **cparams)

In [5]:
cred.get_token("https://database.windows.net/.default").token.encode('utf-16-le')

b'e\x00y\x00J\x000\x00e\x00X\x00A\x00i\x00O\x00i\x00J\x00K\x00V\x001\x00Q\x00i\x00L\x00C\x00J\x00h\x00b\x00G\x00c\x00i\x00O\x00i\x00J\x00S\x00U\x00z\x00I\x001\x00N\x00i\x00I\x00s\x00I\x00n\x00g\x001\x00d\x00C\x00I\x006\x00I\x00k\x00p\x00Z\x00a\x00E\x00F\x00j\x00V\x00F\x00B\x00N\x00W\x00l\x009\x00M\x00W\x00D\x00Z\x00E\x00Q\x00m\x00x\x00P\x00V\x001\x00E\x003\x00S\x00G\x004\x00w\x00T\x00m\x00V\x00Y\x00R\x00S\x00I\x00s\x00I\x00m\x00t\x00p\x00Z\x00C\x00I\x006\x00I\x00k\x00p\x00Z\x00a\x00E\x00F\x00j\x00V\x00F\x00B\x00N\x00W\x00l\x009\x00M\x00W\x00D\x00Z\x00E\x00Q\x00m\x00x\x00P\x00V\x001\x00E\x003\x00S\x00G\x004\x00w\x00T\x00m\x00V\x00Y\x00R\x00S\x00J\x009\x00.\x00e\x00y\x00J\x00h\x00d\x00W\x00Q\x00i\x00O\x00i\x00J\x00o\x00d\x00H\x00R\x00w\x00c\x00z\x00o\x00v\x00L\x002\x00R\x00h\x00d\x00G\x00F\x00i\x00Y\x00X\x00N\x00l\x00L\x00n\x00d\x00p\x00b\x00m\x00R\x00v\x00d\x003\x00M\x00u\x00b\x00m\x00V\x000\x00I\x00i\x00w\x00i\x00a\x00X\x00N\x00z\x00I\x00j\x00o\x00i\x00a\x00H\x00R\x000\x00c\x00H\x00M\x

In [9]:
inject_azure_credential(cred, create_engine(connection_string))

NameError: name 'event' is not defined

In [8]:

engine2 = create_engine(connection_string, connect_args={'timeout': 120})

In [13]:
with engine2.connect() as connection:
    result = connection.execute(text("INSERT INTO [dbo].[new_table] (age, name) VALUES (25, 'Test User')"))
    print(result)

In [12]:
with engine2.connect() as connection:
    result = connection.execute(text("SELECT * from [dbo].[new_table]"))
    print(result.all())

[(36, 'Ignacio')]


In [16]:
metadata = MetaData()

# Define the table structure
user_table = Table(
    '[dbo].[new_table]', metadata,
    Column('age', Integer),
    Column('name', String)
)

# Create the table in the database (if it doesn't exist)
metadata.create_all(engine2)

# Construct the INSERT statement
stmt = insert(user_table).values(age=26, name='alice@example.com')

# Execute the statement
with engine2.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()
    # print(f"Inserted row with ID: {result.inserted_primary_key[0]}")

# # Inserting multiple rows
# data_to_insert = [
#     {'name': 'Bob', 'email': 'bob@example.com'},
#     {'name': 'Charlie', 'email': 'charlie@example.com'}
# ]
# stmt_multiple = insert(user_table).values(data_to_insert)

# with engine.connect() as conn:
#     conn.execute(stmt_multiple)
#     conn.commit()
#     print("Inserted multiple rows.")    

In [ ]:
result.

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [8]:
from msal import ConfidentialClientApplication
import os
TENANT_ID =  os.getenv("AZURE_TENANT_ID")
CLIENT_ID =  os.getenv("AZURE_CLIENT_ID")
CLIENT_SECRET = os.getenv("AZURE_CLIENT_SECRET")

AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
SCOPE = ["https://database.windows.net/.default"]
SCOPE_FABRIC = ["https://api.fabric.microsoft.com/.default"]


app = ConfidentialClientApplication(
    CLIENT_ID,
    authority=AUTHORITY,
    client_credential=CLIENT_SECRET,
)

result = app.acquire_token_for_client(scopes=SCOPE_FABRIC)


In [ ]:
app.get_

In [ ]:
token = credential.get_token(token_url).token.encode('utf-16-le')

In [7]:
result["access_token"]

'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSIsImtpZCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSJ9.eyJhdWQiOiJodHRwczovL2FwaS5mYWJyaWMubWljcm9zb2Z0LmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzAwOGU1NjBmLTA4YWYtNGNlYy1hMDU2LWIzNTQ0NzUwMzk5MS8iLCJpYXQiOjE3NTgxOTI5NTUsIm5iZiI6MTc1ODE5Mjk1NSwiZXhwIjoxNzU4MTk2ODU1LCJhaW8iOiJrMlJnWU5oeHVraFFLdnY5TEsyMlYzNFRQdTdkQ1FBPSIsImFwcGlkIjoiODRkYmM3N2MtNjEzMS00ODJjLWE5OTgtZjM4YzRmOGQ4MGQ0IiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMDA4ZTU2MGYtMDhhZi00Y2VjLWEwNTYtYjM1NDQ3NTAzOTkxLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiMjhhOGU0MGUtNzk2ZS00Zjg0LWJmYTQtMmUwNzFmMTUyYTMxIiwicmgiOiIxLkFUa0FEMWFPQUs4STdFeWdWck5VUjFBNWtRa0FBQUFBQUFBQXdBQUFBQUFBQUFDcUFRQTVBQS4iLCJzdWIiOiIyOGE4ZTQwZS03OTZlLTRmODQtYmZhNC0yZTA3MWYxNTJhMzEiLCJ0aWQiOiIwMDhlNTYwZi0wOGFmLTRjZWMtYTA1Ni1iMzU0NDc1MDM5OTEiLCJ1dGkiOiI3NHBpWXZXbXAwNnlQbDNGbmIwX0FBIiwidmVyIjoiMS4wIiwieG1zX2Z0ZCI6ImdFd0xIRmFWVWVORVJCZTM5VkowQ1lYZ24zTGJheTJPNEwyRGw1Q2VORUlCWm

In [ ]:
import requests
import msal

# 1. Acquire token
app = msal.ConfidentialClientApplication(
   client_id="YOUR_CLIENT_ID",
   client_credential="YOUR_CLIENT_SECRET",
   authority="https://login.microsoftonline.com/YOUR_TENANT_ID"
)
result = app.acquire_token_for_client(scopes=["https://api.fabric.microsoft.com/.default"])
token = result["access_token"]

# 2. Call API
headers = {"Authorization": f"Bearer {token}"}
resp = requests.get("https://api.fabric.microsoft.com/v1/connections", headers=headers)
resp.raise_for_status()